In [47]:
import pandas as pd
import numpy as np

In [49]:
df=pd.read_csv("C:\\Users\\SukilPriya\\Downloads\\archive (3)\\Groceries_dataset.csv")
df.head()

,Member_number,Date,itemDescription
0,1808,21-07-2015,tropical fruit
1,2552,05-01-2015,whole milk
2,2300,19-09-2015,pip fruit
3,1187,12-12-2015,other vegetables
4,3037,01-02-2015,whole milk


In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38765 entries, 0 to 38764
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Member_number    38765 non-null  int64 
 1   Date             38765 non-null  object
 2   itemDescription  38765 non-null  object
dtypes: int64(1), object(2)
memory usage: 908.7+ KB


In [53]:
df.isnull().sum()

Member_number      0
Date               0
itemDescription    0
dtype: int64

In [55]:
# add some time series metrics to the dataframe
df['Date'] = pd.to_datetime(df['Date'])
df['Week'] = df['Date'].dt.isocalendar().week
df['Month'] = df['Date'].dt.month

C:\Users\SukilPriya\AppData\Local\Temp\ipykernel_20148\1248066321.py:2: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['Date'] = pd.to_datetime(df['Date'])


In [57]:
# break the data into lists:
## transaction list
## all items a customer purchased list
## all items a customer purchased in a month list

individual_transactions = [a[1]['itemDescription'].tolist() for a in list(df.groupby(['Member_number','Date']))]
customer_transactions = [a[1]['itemDescription'].tolist() for a in list(df.groupby(['Member_number']))]
customer_month_transactions = [a[1]['itemDescription'].tolist() for a in list(df.groupby(['Member_number', 'Month']))]

In [59]:
print(individual_transactions[:3])

[['whole milk', 'pastry', 'salty snack'], ['sausage', 'whole milk', 'semi-finished bread', 'yogurt'], ['soda', 'pickled vegetables']]


In [41]:
from mlxtend.frequent_patterns import apriori

Note: you may need to restart the kernel to use updated packages.


In [69]:
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
te_array = te.fit(individual_transactions).transform(individual_transactions)

df = pd.DataFrame(te_array,
                  columns=te.columns_)

print(f"rows = {len(df)}")

df.sample(3)

rows = 14963


,Instant food products,UHT-milk,abrasive cleaner,artif. sweetener,baby cosmetics,bags,baking powder,bathroom cleaner,beef,berries,...,turkey,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
13690,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
11565,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4675,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [71]:
from mlxtend.frequent_patterns import apriori
results = apriori(df, min_support=0.001, use_colnames = True)
results['length'] = results['itemsets'].apply(lambda x: len(x))
results['count'] = results['support']*len(df)
results.sample(5)

,support,itemsets,length,count
736,0.002941,"(whipped/sour cream, yogurt)",2,44.0
388,0.002072,"(whipped/sour cream, citrus fruit)",2,31.0
299,0.001002,"(tropical fruit, butter milk)",2,15.0
387,0.002874,"(tropical fruit, citrus fruit)",2,43.0
133,0.006683,(spread cheese),1,100.0


In [73]:
results[results['length'] > 1].sort_values(by='support', ascending = False)

,support,itemsets,length,count
625,0.014837,"(whole milk, other vegetables)",2,222.0
677,0.013968,"(rolls/buns, whole milk)",2,209.0
717,0.011629,"(soda, whole milk)",2,174.0
740,0.011161,"(yogurt, whole milk)",2,167.0
609,0.010559,"(rolls/buns, other vegetables)",2,158.0
...,...,...,...,...
670,0.001002,"(rolls/buns, soft cheese)",2,15.0
516,0.001002,"(pastry, hamburger meat)",2,15.0
692,0.001002,"(tropical fruit, salty snack)",2,15.0
202,0.001002,"(chocolate, bottled beer)",2,15.0


In [75]:
results[results['length'] > 2].sort_values(by='support', ascending = False)

,support,itemsets,length,count
749,0.001470,"(sausage, yogurt, whole milk)",3,22.0
747,0.001337,"(yogurt, rolls/buns, whole milk)",3,20.0
742,0.001203,"(other vegetables, rolls/buns, whole milk)",3,18.0
741,0.001136,"(soda, rolls/buns, other vegetables)",3,17.0
743,0.001136,"(soda, whole milk, other vegetables)",3,17.0
744,0.001136,"(yogurt, whole milk, other vegetables)",3,17.0
745,0.001136,"(sausage, rolls/buns, whole milk)",3,17.0
748,0.001069,"(sausage, soda, whole milk)",3,16.0
746,0.001002,"(soda, rolls/buns, whole milk)",3,15.0


In [79]:
from mlxtend.frequent_patterns import association_rules
lift_results = association_rules(results, metric="support", min_threshold=0)
lift_results = lift_results[['antecedents',
                             'consequents',
                             'support',
                             'confidence',
                             'lift']]

lift_results.sample(3)

,antecedents,consequents,support,confidence,lift
632,(pastry),(frankfurter),0.001738,0.033592,0.889616
869,(root vegetables),(newspapers),0.002540,0.036503,0.938488
119,(bottled beer),(fruit/vegetable juice),0.001671,0.036873,1.083955


In [81]:
highest_lift = lift_results[lift_results["lift"] == lift_results["lift"].max()]
print(highest_lift)

               antecedents           consequents  support  confidence  \
1234  (yogurt, whole milk)             (sausage)  0.00147    0.131737   
1235             (sausage)  (yogurt, whole milk)  0.00147    0.024363   

          lift  
1234  2.182917  
1235  2.182917  


In [85]:
product = 'whole milk'
product_subset = lift_results[lift_results['antecedents'] == frozenset({product})]
product_subset.sort_values(by='lift', ascending = False)

,antecedents,consequents,support,confidence,lift
1237,(whole milk),"(sausage, yogurt)",0.001470,0.009310,1.619866
1213,(whole milk),"(sausage, rolls/buns)",0.001136,0.007194,1.345594
1231,(whole milk),"(sausage, soda)",0.001069,0.006771,1.138374
1109,(whole milk),(semi-finished bread),0.001671,0.010580,1.114825
1225,(whole milk),"(yogurt, rolls/buns)",0.001337,0.008464,1.082428
...,...,...,...,...,...
1088,(whole milk),(salty snack),0.001938,0.012273,0.653502
578,(whole milk),(dessert),0.002406,0.015235,0.645777
302,(whole milk),(butter milk),0.001671,0.010580,0.601921
840,(whole milk),(misc. beverages),0.001403,0.008887,0.563459
